## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

#### Best Practises
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW,TFIDF,Word2vec
4. Train ML algorithms

In [ ]:
## load the dataset
import pandas as pd
data = pd.read_csv('/content/all_kindle_review.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [ ]:
df=data[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [ ]:
df.shape

(12000, 2)

In [ ]:
## Missing Values
df.isnull().sum()

,0
reviewText,0
rating,0


In [ ]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [ ]:
df['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


# PREPROCESSING AND CLEANING

In [ ]:
## positive review is 1 and negative review is 0
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

<ipython-input-7-eec9d39b699f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)


In [ ]:
df['rating'].value_counts()

,count
rating,
1,8000
0,4000


In [ ]:
## 1. Lower All the cases
df['reviewText']=df['reviewText'].str.lower()

<ipython-input-9-26e5fcb41736>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


In [ ]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

<ipython-input-13-86b4aeb70b16>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
<ipython-input-13-86b4aeb70b16>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
<ipython-input-13-86b4aeb70b16>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [ ]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [ ]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

<ipython-input-18-7fa46fdadeba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))


In [ ]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [ ]:
nltk.download('punkt_tab')
!pip install gensim

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
data = df['reviewText']

In [ ]:
words=[]
for sent in df['reviewText']:
  sent_token = sent_tokenize(sent)
  for sent in sent_token:
    words.append(simple_preprocess(sent))

In [ ]:
words

[['jace',
  'rankin',
  'may',
  'short',
  'he',
  'nothing',
  'mess',
  'man',
  'hauled',
  'saloon',
  'undertaker',
  'know',
  'he',
  'famous',
  'bounty',
  'hunter',
  'oregon',
  'shot',
  'man',
  'saloon',
  'finished',
  'year',
  'long',
  'quest',
  'avenge',
  'sister',
  'murder',
  'trying',
  'figure',
  'next',
  'snotty',
  'nosed',
  'farm',
  'boy',
  'rescued',
  'gang',
  'bully',
  'offer',
  'money',
  'kill',
  'man',
  'forced',
  'ranch',
  'reluctantly',
  'agrees',
  'bring',
  'man',
  'justice',
  'kill',
  'outright',
  'first',
  'need',
  'tell',
  'sister',
  'widower',
  'newskyla',
  'kyle',
  'springer',
  'bailey',
  'riding',
  'trail',
  'sleeping',
  'ground',
  'past',
  'month',
  'trying',
  'find',
  'jace',
  'want',
  'revenge',
  'man',
  'killed',
  'husband',
  'took',
  'ranch',
  'amongst',
  'crime',
  'shes',
  'keen',
  'detour',
  'jace',
  'want',
  'take',
  'realizes',
  'shes',
  'option',
  'hide',
  'behind',
  'boy',
 

# TRAINING WORD TO VEC

In [ ]:
import gensim

In [ ]:
model = gensim.models.Word2Vec(words)

In [ ]:
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'series',
 'well',
 'much',
 'first',
 'even',
 'didnt',
 'short',
 'know',
 'way',
 'great',
 'could',
 'make',
 'sex',
 'little',
 'dont',
 'two',
 'thing',
 'want',
 'think',
 'find',
 'plot',
 'romance',
 'also',
 'end',
 'life',
 'im',
 'see',
 'enjoyed',
 'go',
 'scene',
 'never',
 'written',
 'take',
 'woman',
 'many',
 'lot',
 'kindle',
 'year',
 'say',
 'thought',
 'work',
 'bit',
 'found',
 'going',
 'give',
 'interesting',
 'liked',
 'writing',
 'novel',
 'loved',
 'another',
 'feel',
 'better',
 'got',
 'come',
 'man',
 'hot',
 'still',
 'back',
 'enough',
 'though',
 'people',
 'star',
 'reader',
 'made',
 'something',
 'review',
 'part',
 'friend',
 'page',
 'cant',
 'bad',
 'world',
 'need',
 'free',
 'new',
 'keep',
 'wasnt',
 'doesnt',
 'relationship',
 'enjoy',
 'recommend',
 'together',
 'next',
 'felt',
 'start',
 'best',
 'put',

In [ ]:
model.corpus_count

12000

In [ ]:
model.epochs

5

In [ ]:
model.wv.similar_by_word('amazing')

[('exciting', 0.9483686089515686),
 ('fantastic', 0.9386157393455505),
 ('captivating', 0.928534209728241),
 ('continuation', 0.9246478080749512),
 ('memorable', 0.9202097058296204),
 ('awesome', 0.9197221398353577),
 ('compelling', 0.9109065532684326),
 ('absolutely', 0.9074524641036987),
 ('satisfying', 0.9031193256378174),
 ('draw', 0.9026501178741455)]

In [ ]:
model.wv['superb']

array([-0.06660758,  0.1659932 ,  0.0868527 , -0.01925161,  0.06955788,
       -0.23142543,  0.11919067,  0.31917483, -0.05628079, -0.09201039,
       -0.05387443, -0.25085753, -0.10158101,  0.06886955,  0.06624451,
       -0.15898791,  0.02838592, -0.10779949, -0.05115189, -0.2919231 ,
        0.12800221,  0.1142711 ,  0.02968161, -0.05545193, -0.06912984,
        0.02955392, -0.04533562, -0.04433395, -0.12867633, -0.01154781,
        0.13591632,  0.00567745,  0.0435465 , -0.14297736, -0.04835919,
        0.09165197,  0.04505217, -0.04268537, -0.1592501 , -0.13400324,
        0.07694764, -0.10476657, -0.17810662, -0.00256337,  0.13172111,
        0.09256484, -0.1136919 , -0.05486565,  0.12018976,  0.17025447,
        0.09202147, -0.18271077, -0.05300977, -0.06006525, -0.14144918,
        0.10539953,  0.1130994 , -0.02336459, -0.0917594 ,  0.1062243 ,
        0.05026928, -0.01465224, -0.00541895,  0.01859017, -0.10150248,
        0.11172745,  0.08248937,  0.03537089, -0.19102472,  0.21

In [ ]:
model.wv['good'].shape

(100,)

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
def avg_word2vec(doc):
  vec = [model.wv[word] for word in doc if word in model.wv.index_to_key]
  return np.mean(vec,axis=0)

In [ ]:
import numpy as np
X = []
for i  in tqdm(range(len(words))):
  X.append(avg_word2vec(words[i]))

100%|██████████| 12000/12000 [00:33<00:00, 363.07it/s]


In [ ]:
len(X)

12000

In [ ]:
df.shape

(12000, 2)

In [ ]:
X # vector for each word

[array([-0.19415875,  0.28402364,  0.05793992,  0.15672661,  0.02551155,
        -0.4039793 ,  0.15159354,  0.5600478 , -0.25977486, -0.17398223,
        -0.30439776, -0.27316648, -0.04278212,  0.19025849,  0.03490169,
        -0.21187608,  0.11496274, -0.299643  ,  0.00438215, -0.50205094,
         0.02967746,  0.01210497,  0.1671544 , -0.18145183,  0.09649726,
        -0.00886706, -0.23250021, -0.14226297, -0.3057275 ,  0.07690768,
         0.33308154,  0.07924125,  0.21487546, -0.05857334, -0.1776808 ,
         0.52187794,  0.12709183, -0.17274204,  0.04445826, -0.66073036,
        -0.11112989, -0.26776564, -0.11573412,  0.02535843,  0.1938432 ,
        -0.25604835, -0.20530546, -0.11497109,  0.19810444,  0.2230267 ,
         0.27746344, -0.26215914, -0.03853406,  0.17943032, -0.43069884,
         0.24796195,  0.1600885 , -0.0440633 , -0.4253137 ,  0.20307416,
         0.07754804,  0.07201669, -0.04590723, -0.20073575, -0.24169931,
         0.3302391 ,  0.16492465,  0.2110181 , -0.3

In [ ]:
X[1]

array([-0.21285227,  0.17499384, -0.12462614,  0.31073704,  0.3828968 ,
       -0.5396461 ,  0.4114976 ,  0.7435964 , -0.30372286, -0.2648884 ,
       -0.2930616 , -0.27077803,  0.13131925,  0.4526464 ,  0.3887558 ,
       -0.37550497,  0.48556817, -0.13913979, -0.2097059 , -1.2088623 ,
        0.05003221,  0.1527033 ,  0.08063686, -0.43560842,  0.05459024,
       -0.03522917, -0.3626829 , -0.20871733, -0.4324668 ,  0.1523516 ,
        0.33409822,  0.03712254,  0.45561707, -0.3400634 , -0.6419402 ,
        0.7842649 , -0.0779013 , -0.35507703, -0.21336797, -0.37729806,
        0.30077526, -0.21699516, -0.3450091 ,  0.03246881,  0.3780179 ,
       -0.01446325, -0.42877963, -0.38261262,  0.34562853,  0.4121313 ,
        0.52114695, -0.38087142, -0.34243485,  0.27725762, -0.27015117,
        0.3059873 ,  0.36195222,  0.0176005 , -0.59217894,  0.2100036 ,
        0.05754874, -0.35364094,  0.45550603, -0.12308025, -0.6248606 ,
        0.5988064 ,  0.26332292,  0.07557017, -0.64540243,  0.57

In [ ]:
X[1].shape

(100,)

In [ ]:
##independent Features
X=np.array(X)

In [ ]:
X

array([[-0.19415875,  0.28402364,  0.05793992, ..., -0.2769195 ,
         0.0023525 , -0.09034883],
       [-0.21285227,  0.17499384, -0.12462614, ..., -0.12353417,
         0.00307193, -0.09117939],
       [-0.14922158,  0.3098691 ,  0.09309042, ..., -0.12286097,
        -0.11779602, -0.02323611],
       ...,
       [-0.24828869,  0.20072006, -0.2040986 , ..., -0.01570402,
         0.02654998, -0.07330859],
       [ 0.06230992,  0.38908097,  0.06619278, ..., -0.5246986 ,
         0.25238112, -0.18674909],
       [-0.2585687 ,  0.24006549, -0.00889509, ..., -0.1068553 ,
         0.04797799, -0.16134207]], dtype=float32)

In [ ]:
# Accumulate reshaped arrays in a list
rows = [pd.DataFrame(X[i].reshape(1, -1)) for i in range(len(X))]

# Concatenate all DataFrames at once
df2 = pd.concat(rows, ignore_index=True)

In [ ]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.194159,0.284024,0.057940,0.156727,0.025512,-0.403979,0.151594,0.560048,-0.259775,-0.173982,...,0.375940,0.110350,-0.001436,0.000895,0.657184,0.273147,-0.029777,-0.276920,0.002352,-0.090349
1,-0.212852,0.174994,-0.124626,0.310737,0.382897,-0.539646,0.411498,0.743596,-0.303723,-0.264888,...,0.676609,0.168192,-0.224610,-0.083678,0.361112,0.295593,0.200340,-0.123534,0.003072,-0.091179
2,-0.149222,0.309869,0.093090,0.156891,0.246335,-0.549708,0.330785,0.766370,-0.268945,-0.248805,...,0.493953,0.118054,-0.058269,-0.123229,0.379593,0.273225,0.136035,-0.122861,-0.117796,-0.023236
3,-0.360788,0.321756,0.145234,0.201940,0.288696,-0.597947,0.251742,0.804438,-0.023198,-0.087265,...,0.450072,0.103020,-0.053708,-0.340314,-0.018014,0.136429,0.404604,0.124946,-0.135871,0.004962
4,0.195935,0.390909,-0.032445,0.005642,0.071651,-0.495023,0.305871,0.874430,-0.246464,-0.301050,...,0.674629,0.072959,0.041017,-0.166935,0.457544,0.259000,0.160233,-0.241403,0.016762,-0.189343


In [ ]:
df2.isnull().sum()

,0
0,0
1,0
2,0
3,0
4,0
...,...
95,0
96,0
97,0
98,0


In [ ]:
# Dependent features
y = df.iloc[:,-1]

In [ ]:
y

,rating
0,1
1,1
2,1
3,1
4,1
...,...
11995,1
11996,1
11997,1
11998,0


In [ ]:
df2['output'] = y

In [ ]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,output
0,-0.194159,0.284024,0.057940,0.156727,0.025512,-0.403979,0.151594,0.560048,-0.259775,-0.173982,...,0.110350,-0.001436,0.000895,0.657184,0.273147,-0.029777,-0.276920,0.002352,-0.090349,1
1,-0.212852,0.174994,-0.124626,0.310737,0.382897,-0.539646,0.411498,0.743596,-0.303723,-0.264888,...,0.168192,-0.224610,-0.083678,0.361112,0.295593,0.200340,-0.123534,0.003072,-0.091179,1
2,-0.149222,0.309869,0.093090,0.156891,0.246335,-0.549708,0.330785,0.766370,-0.268945,-0.248805,...,0.118054,-0.058269,-0.123229,0.379593,0.273225,0.136035,-0.122861,-0.117796,-0.023236,1
3,-0.360788,0.321756,0.145234,0.201940,0.288696,-0.597947,0.251742,0.804438,-0.023198,-0.087265,...,0.103020,-0.053708,-0.340314,-0.018014,0.136429,0.404604,0.124946,-0.135871,0.004962,1
4,0.195935,0.390909,-0.032445,0.005642,0.071651,-0.495023,0.305871,0.874430,-0.246464,-0.301050,...,0.072959,0.041017,-0.166935,0.457544,0.259000,0.160233,-0.241403,0.016762,-0.189343,1


In [ ]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33)

In [101]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [102]:
classifier=RandomForestClassifier()
nb = GaussianNB()
reg = LogisticRegression(max_iter=1000)
svm = SVC()

In [103]:
models = [classifier,nb,reg,svm]

In [104]:
for model in models:
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)

  print("Accuracy score for ",model," ",accuracy_score(y_test,y_pred))
  print("classification report \n",classification_report(y_test,y_pred))
  print("-"*60)


Accuracy score for  RandomForestClassifier()   0.7707070707070707
classification report 
               precision    recall  f1-score   support

           0       0.67      0.58      0.62      1287
           1       0.81      0.86      0.84      2673

    accuracy                           0.77      3960
   macro avg       0.74      0.72      0.73      3960
weighted avg       0.76      0.77      0.77      3960

------------------------------------------------------------
Accuracy score for  GaussianNB()   0.7131313131313132
classification report 
               precision    recall  f1-score   support

           0       0.54      0.72      0.62      1287
           1       0.84      0.71      0.77      2673

    accuracy                           0.71      3960
   macro avg       0.69      0.71      0.69      3960
weighted avg       0.74      0.71      0.72      3960

------------------------------------------------------------
Accuracy score for  LogisticRegression(max_iter=1000)   